In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
metadata_path = './skin-cancer-mnist-ham10000/HAM10000_metadata.csv'
metadat = pd.read_csv(metadata_path)
num_samples = len(metadat)
metadat = metadat.sample(frac=1,random_state=12).reset_index(drop=True) #shuffling data

#metadat = metadat['image_id'].add('.jpg')
metadat.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0003559,ISIC_0025810,bkl,consensus,75.0,male,face
1,HAM_0004240,ISIC_0032517,mel,histo,50.0,male,back
2,HAM_0002610,ISIC_0026876,vasc,consensus,0.0,female,abdomen
3,HAM_0003229,ISIC_0031140,bcc,histo,60.0,male,chest
4,HAM_0005963,ISIC_0028129,nv,follow_up,45.0,male,foot


In [3]:
# Use 20% test split (80% training + validation)
ntrain2 = int(len(metadat)*0.8)
ntrain1 = int(ntrain2*0.8)
x_test = metadat.iloc[ntrain2:,:]
x_train = metadat.iloc[:ntrain1,:]
x_val = metadat.iloc[ntrain1:ntrain2,:]

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(299, 299),
        batch_size=32,
        seed=12)

validation_generator = test_datagen.flow_from_directory(
        './val',
        target_size=(299, 299),
        batch_size=32,
        seed=12)


Found 6409 images belonging to 7 classes.
Found 1602 images belonging to 7 classes.


In [4]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 299, 299
train_data_dir = "./train"
validation_data_dir = "./val"
nb_train_samples = 6409
nb_validation_samples = 1602 
batch_size = 16
epochs = 50

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))


# Freeze the layers which you don't want to train. Here I am freezing the all layers.
for layer in model.layers:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(7, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\Raul\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 6409 images belonging to 7 classes.
Found 1602 images belonging to 7 classes.
Instructions for updating:
Use tf.cast instead.


C:\Users\Raul\Anaconda3\lib\site-packages\ipykernel_launcher.py:80: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\Raul\Anaconda3\lib\site-packages\ipykernel_launcher.py:80: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=50, validation_data=<keras_pre..., callbacks=[<keras.ca..., steps_per_epoch=400, validation_steps=1602)`


Epoch 1/50


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node block1_conv1/convolution}}]]
	 [[{{node loss/mul}}]]

In [ ]:
model.save('./models/vgg_epoch50.h5')

In [ ]:
for i, layer in enumerate(model_final.layers):
   print(i, layer.name)

In [ ]:
from keras.metrics import binary_accuracy
y_pred = model.predict_generator(validation_generator)
y_pred = np.argmax(y_pred,axis=0)
count = 0
correct = 0
for i in range(len(validation_generator)):
    if np.argmax(validation_generator[i][1]) == y_pred[i]:
        correct +=1
    count+=1
    print(str(acc))